# AmebaPro2 pyNgrok Whisper Server

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install fastapi uvicorn
!pip install pyngrok

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-r6gmmmt8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-r6gmmmt8
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import whisper
ASR = whisper.load_model("base").to("cpu")

## HTTP Server with Ngrok

In [3]:
import getpass
import os
import threading

from pyngrok import ngrok, conf

## set ngrok authtoken
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
········


In [4]:
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


 * ngrok tunnel "https://40d9-2407-4d00-8d00-00-f8.ngrok-free.app" -> "http://127.0.0.1:5000/"


In [5]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import Response
from pydantic import BaseModel
import uvicorn
import base64

app = FastAPI()

class Base64Data(BaseModel):
    base64_string: str

@app.get("/")
def root():
    return Response("Hello World!")

@app.post("/audio")
async def post_audio(data: Base64Data ):
    try:
        #Decode the base64 string
        decoded_data = base64.b64decode(data.base64_string)

        # print(decoded_data)
        #Save the decoded data to an MP4 file
        with open("output.mp4", "wb") as f:
            f.write(decoded_data)

        # Whisper transcribe
        result = ASR.transcribe("output.mp4")
        print("ASR: "+result["text"])

        return Response(result["text"])
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# start new thread
threading.Thread(uvicorn.run(app, host="127.0.0.1", port=5000, log_level="info")).start()

INFO:     Started server process [24512]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)
